In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cufflinks as cf
import statsmodels.api as sm
import plotly.graph_objects as go
import numpy as np
from pandas_datareader import data as wb
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

/Users/danvaldes/anaconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

Pandas display options to supress scientifc notation and show all columns and all rows if necessary. Puts a lot of strain of processor

In [ ]:
# pd.options.display.max_columns = None
# pd.options.display.max_rows = None
pd.options.display.float_format= '{:.2f}'.format

In [2]:
nyse = pd.read_csv(os.path.join('Resources', 'nyse.csv'))
nasdaq = pd.read_csv(os.path.join('Resources', 'nasdaq.csv'))
amex = pd.read_csv(os.path.join('Resources', 'amex.csv'))

frames = [nyse, nasdaq, amex]
tickers = pd.concat(frames).sort_values(by='Ticker')

In [3]:
data_2018 = pd.read_csv(os.path.join('Resources', '2018_Financial_Data.csv')).\
rename(columns={'Unnamed: 0':'Ticker'}).\
sort_values(by='Ticker').\
reset_index(drop=True)

df = tickers.merge(data_2018, on='Ticker')

In [4]:
df = df[df['Market Cap'].notna()];

In [6]:
df['Market Cap Size'] = ['Large Cap' if x > 10000000000 else 'Mid Cap' if x > 2000000000 else 'Small Cap' for x in df['Market Cap']]

Note: In *__data_2018__* we had 4392 observations. When we merged with our *__tickers__* dataframe which had 8000+ tickers and company names from exchanges we lost 150 observations. These can be retrieved with 
``` 
data_2018[~data_2018.Ticker.isin(tickers['Ticker'])]
```
Here the **~** symbol essentially means NOT IN when we filter.\
Lastly, a dataframe with just indicators we can manipulate. We can filter out some of the columns containing information we don't need then pass this as a filter to our working dataframe with ```df[x['Indicator']]```

We can create a dataframe that counts the null values across each column to see what we can begin to drop. We can choose some threshold for amount of null values that once met causes us to drop the column. If we choose 500, we trim out about 30 columns. 

In [8]:
null_count = pd.DataFrame(df.isna().sum().sort_values(ascending=False))

In [9]:
null_count;

In [34]:
null_count[null_count[0] >= 500];

In [13]:
to_drop = null_count[null_count[0] >= 500].index

In [14]:
df = df.drop(columns=to_drop)

In [16]:
df.head()

,Ticker,Company Name,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,...,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,Sector,2019 PRICE VAR [%],Class,Market Cap Size
0,A,Agilent Technologies,4.914000e+09,0.0988,2.227000e+09,2.687000e+09,385000000.0,1.374000e+09,1.759000e+09,9.280000e+08,...,0.1096,0.0136,-0.0517,-0.1054,0.1357,0.1180,Healthcare,30.976102,1,Large Cap
1,AA,Alcoa Corp,1.340300e+10,0.1503,1.008100e+10,3.322000e+09,31000000.0,2.480000e+08,1.603000e+09,1.719000e+09,...,0.2519,-0.0754,0.2323,0.2835,-0.0313,-0.1143,Basic Materials,-18.025913,0,Mid Cap
2,AAL,American Airlines Gp,4.454100e+10,0.0450,2.040000e+10,2.414100e+10,0.0,1.377100e+10,2.148500e+10,2.656000e+09,...,0.1199,0.1477,0.7748,0.3576,0.0000,0.0253,Industrials,-10.602069,0,Large Cap
3,AAMC,Altisource Asset,1.592600e+07,-0.1230,0.000000e+00,1.592600e+07,0.0,2.253400e+07,2.253400e+07,-6.608000e+06,...,0.0000,-0.1825,-0.0253,0.0000,0.0000,-0.1166,Financial Services,-58.847050,0,Small Cap
4,AAME,Atlantic Amer Cp,1.855530e+08,0.0245,1.326500e+08,5.290300e+07,0.0,3.904200e+07,5.183700e+07,1.066000e+06,...,0.0000,0.0030,-0.0921,0.0000,0.0000,-0.1014,Financial Services,-20.250605,0,Small Cap


If we count the null values across rows, we see that some companies are missing info on 150+ columns. We set the threshold to be 120, meaning rows with less than 120 populated columns are dropped.

In [19]:
row_null_count = pd.DataFrame(df.isnull().sum(axis = 1).sort_values(ascending=False))

In [26]:
row_null_count.head(5)

,0
1153,106
1819,104
220,103
1219,101
3862,98


In [27]:
df = df.dropna(thresh=120)

Finally we fill null values with the means across cap size

In [28]:
small_caps = df[df['Market Cap Size'] == 'Small Cap']
mid_caps = df[df['Market Cap Size'] == 'Mid Cap']
large_caps = df[df['Market Cap Size'] == 'Large Cap']

In [37]:
small_caps = df[df['Market Cap Size'] == 'Small Cap'].fillna(small_caps.mean())
mid_caps = df[df['Market Cap Size'] == 'Mid Cap'].fillna(mid_caps.mean())
large_caps = df[df['Market Cap Size'] == 'Large Cap'].fillna(large_caps.mean())

In [43]:
caps = [small_caps, mid_caps, large_caps]

market = pd.concat(caps)

market;

In [ ]:
indicators = pd.DataFrame({
   'Indicator': market.columns.tolist()
})

filt = indicators[indicators['Indicator'].str.contains('Growth|Margin|ratio|Ratio|Turnover|per|/| / | to |Per|return|To|days') == False]

In [ ]:
cond = x['Indicator'].to_list()

In [44]:
# market.to_csv(os.path.join('cleaned_data.csv'))

## Conditional Filtering

In [ ]:
tickers[tickers['Ticker'].str.startswith('BA') == True];

In [ ]:
tickers[tickers['Company Name'].str.contains('ETF') == True];

In [ ]:
df[df['Company Name'].str.contains('ETF') == True];

In [ ]:
df[df['Ticker'].str.startswith('BA') == True];

In [ ]:
df[df['Revenue Growth'] > 10].sort_values(by='Revenue', ascending=False).head()

# Iterators 

Sector breakdown across Market Cap Sizes

In [ ]:
count = pd.DataFrame(df.groupby(['Market Cap Size'])['Sector'].value_counts());

count_gen = count.groupby(['Market Cap Size']).__iter__()

In [ ]:
count_group_id, count_grouped_data = count_gen.__next__()
print(count_group_id) 
count_grouped_data

Iterate across Market Caps and Sectors

In [ ]:
mcap_gen = df.groupby(['Market Cap Size']).__iter__()
sector_gen = df.groupby(['Sector']).__iter__()

In [ ]:
market_group_id, market_grouped_data = mcap_gen.__next__()
print(market_group_id) 
market_grouped_data;

In [ ]:
sector_group_id, sector_grouped_data = sector_gen.__next__()
print(sector_group_id)
sector_grouped_data;

In [ ]:
scap_gen = small_caps.groupby(['Sector']).__iter__()

In [ ]:
scap_group_id, scap_grouped_data = scap_gen.__next__()
print(scap_group_id) 
scap_grouped_data.sort_values(by='Revenue Growth', ascending=False)

In [ ]:
assets = scap_grouped_data['Ticker'].to_list()

pf_data = pd.DataFrame()

for a in assets:
    pf_data[a] = wb.DataReader(a, data_source = 'yahoo', start = '2018-1-1')['Adj Close']


In [ ]:
mcap_gen = mid_caps.groupby(['Sector']).__iter__()

In [ ]:
mcap_group_id, mcap_grouped_data = mcap_gen.__next__()
print(mcap_group_id) 
mcap_grouped_data.sort_values(by='Revenue Growth', ascending=False)

In [ ]:
assets = mcap_grouped_data['Ticker'].to_list()

pf_data = pd.DataFrame()

for a in assets:
    pf_data[a] = wb.DataReader(a, data_source = 'yahoo', start = '2018-1-1')['Adj Close']


In [ ]:
lcap_gen = large_caps.groupby(['Sector']).__iter__()

In [ ]:
lcap_group_id, lcap_grouped_data = lcap_gen.__next__()
print(lcap_group_id) 
lcap_grouped_data.sort_values(by='Revenue Growth', ascending=False)

In [ ]:
assets = lcap_grouped_data['Ticker'].to_list()

pf_data = pd.DataFrame()

for a in assets:
    pf_data[a] = wb.DataReader(a, data_source = 'yahoo', start = '2018-1-1')['Adj Close']

(pf_data / pf_data.iloc[0] * 100).iplot(asFigure=True)

## Graphs and Visualizations

Count of Companies by Sector

In [ ]:
plt.figure(figsize=(18,8))
sns.countplot(x='Sector', data = df, palette='viridis', order=df['Sector'].value_counts().index).set_xticklabels(labels = df['Sector'].value_counts().index, fontdict={'fontsize': 12},rotation=30)
plt.tight_layout()
plt.savefig(os.path.join('Figures','StockVolume_Sector.png'))

Revenues by Sector

In [ ]:
sector_rev = pd.DataFrame(df.groupby(['Sector']).mean()['Revenue']).reset_index().sort_values(by='Revenue', ascending=False)

plt.figure(figsize=(18,8))
plt.title('Revenues by Sector')
sns.barplot(x = sector_rev['Sector'], y = sector_rev['Revenue'], palette='viridis',).set_xticklabels(labels = sector_rev['Sector'], fontdict={'fontsize': 12}, rotation=30)
plt.tight_layout();
plt.savefig(os.path.join('Figures','SectorRev.png'))

Count of Companies across Market Cap Sizes

In [ ]:
plt.figure(figsize=(18,8))
sns.countplot(x='Market Cap Size', data = df, palette='viridis', order=df['Market Cap Size'].value_counts().index).set_xticklabels(labels = df['Market Cap Size'].value_counts().index, fontdict={'fontsize': 12},rotation=30)
plt.tight_layout()
plt.ylabel('Company Count')
plt.savefig(os.path.join('Figures','MarketCapBreakdown.png'))

In [ ]:
bars = []
sectors = df['Sector'].unique()
caps=['Small Cap', 'Mid Cap', 'Large Cap']
for sector in sectors:
    bars.append(go.Bar(name=sector, x=caps, y=np.array(df[df['Sector'] == sector]['Market Cap Size'].value_counts())))

plt.figure(figsize=(30,12))
fig = go.Figure(data=bars)
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [ ]:
sns.pairplot(large_caps[['2019 PRICE VAR [%]', 'Revenue', 'R&D Expenses', 'Sector','Free Cash Flow','Net Debt']], diag_kind="kde")

In [ ]:
corrs = market[cond].corr()

plt.figure(figsize=(20,20))
g = sns.heatmap(corrs, center=0)
plt.title('Correlations Among Corporate Financial Indicators')
plt.savefig(os.path.join('Figures','correlations.png'))

In [ ]:
px.scatter(large_caps, x='Free Cash Flow', y='2019 PRICE VAR [%]', hover_data=['Company Name','Ticker','Market Cap','Sector'])

## Linear Regression Analysis

In [ ]:
y = market["2019 PRICE VAR [%]"]

X = sm.add_constant(market[cond].iloc[:,0:-2])

In [ ]:
reg = sm.OLS(y, X).fit()

reg.summary();